In [1]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import pickle
import optuna
import re
import gc
import time

from contextlib import contextmanager

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_moons
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules
from Baseline import Baseline
from FeatureEngineering2 import FeatureEngineering2
from FeatureEngineering import FeatureEngineering
from models import Model1Lgb, Model1Xgb, Model1NN, Model2Linear

%matplotlib inline
gc.enable()

print("OK\n")

OK



In [2]:
baseline = Baseline('SK_ID_CURR')
modules = Modules('SK_ID_CURR')
FeatureEngineering = FeatureEngineering('SK_ID_CURR')
FeatureEngineering2 = FeatureEngineering2('SK_ID_CURR')

In [3]:
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

In [4]:
# 学習データに対する「目的変数を知らない」予測値と、テストデータに対する予測値を返す関数
def predict_cv(model, train_x, train_y, test_x):
    preds = []
    preds_test = []
    va_idxes = []

    kf = KFold(n_splits=4, shuffle=True, random_state=71)

    # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        model.fit(tr_x, tr_y, va_x, va_y)
        pred = model.predict(va_x)
        print(pred)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)

    # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    # テストデータに対する予測値の平均をとる
    preds_test = np.mean(preds_test, axis=0)

    return pred_train, preds_test

In [5]:
# Display/plot feature importance
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:100].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(15, 20))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances01.png')

In [6]:
def application_train():

    df = pd.read_csv('./home-credit-default-risk/application_train.csv')
    test_df = pd.read_csv('./home-credit-default-risk/application_test.csv')

    df = df.append(test_df).reset_index()
    df = df[df['CODE_GENDER'] != 'XNA']

    lbe = LabelEncoder()

    for col in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
            df[col] = lbe.fit_transform(df[col])

    df = pd.get_dummies(df, dummy_na = True)

    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace = True)
    df['NEW_DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['NEW_INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    df['NEW_INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['NEW_ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['NEW_PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']

    df.drop("index", axis = 1, inplace =  True)

    df.columns = pd.Index(["APP_" + col for col in df.columns.tolist()])

    df.rename(columns={"APP_SK_ID_CURR":"SK_ID_CURR"}, inplace = True)

    df.rename(columns={"APP_TARGET":"TARGET"}, inplace = True)
    
    return df

In [7]:
def bureau_bb():

    #bureau_balance tablosunun okutulması

    bb = pd.read_csv('./home-credit-default-risk/bureau_balance.csv')
    bb = pd.get_dummies(bb, dummy_na = True)

    agg_list = {"MONTHS_BALANCE":"count",
                "STATUS_0":["sum","mean"],
                "STATUS_1":["sum"],
                "STATUS_2":["sum"],
                "STATUS_3":["sum"],
                "STATUS_4":["sum"],
                "STATUS_5":["sum"],
                "STATUS_C":["sum","mean"],
                "STATUS_X":["sum","mean"] }

    bb_agg = bb.groupby("SK_ID_BUREAU").agg(agg_list)

    # Degisken isimlerinin yeniden adlandirilmasi 
    bb_agg.columns = pd.Index([col[0] + "_" + col[1].upper() for col in bb_agg.columns.tolist()])

    # Status_sum ile ilgili yeni bir degisken olusturma
    bb_agg['NEW_STATUS_SCORE'] = bb_agg['STATUS_1_SUM'] + bb_agg['STATUS_2_SUM']^2 + bb_agg['STATUS_3_SUM']^3 + bb_agg['STATUS_4_SUM']^4 + bb_agg['STATUS_5_SUM']^5

    bb_agg.drop(['STATUS_1_SUM','STATUS_2_SUM','STATUS_3_SUM','STATUS_4_SUM','STATUS_5_SUM'], axis=1,inplace=True)

    bureau = pd.read_csv('./home-credit-default-risk/bureau.csv')
    bureau_and_bb = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')

    #BUREAU BALANCE VE BUREAU ORTAK TABLO

    #CREDIT_TYPE degiskeninin sinif sayisini 3'e düsürmek 
    bureau_and_bb['CREDIT_TYPE'] = bureau_and_bb['CREDIT_TYPE'].replace(['Car loan',
              'Mortgage',
              'Microloan',
              'Loan for business development', 
              'Another type of loan',
              'Unknown type of loan', 
              'Loan for working capital replenishment',
              "Loan for purchase of shares (margin lending)",                                                
              'Cash loan (non-earmarked)', 
              'Real estate loan',
              "Loan for the purchase of equipment", 
              "Interbank credit", 
              "Mobile operator loan"], 'Rare')


    #CREDIT_ACTIVE degiskeninin sinif sayisini 2'ye düsürmek (Sold' u Closed a dahil etmek daha mi uygun olur ???)
    bureau_and_bb['CREDIT_ACTIVE'] = bureau_and_bb['CREDIT_ACTIVE'].replace(['Bad debt','Sold'], 'Active')

    # bureau_bb tablosundaki kategorik degiskenlere One Hot Encoding uygulanmasi
    bureau_and_bb = pd.get_dummies(bureau_and_bb, columns = ["CREDIT_TYPE","CREDIT_ACTIVE"])

    # CREDIT_CURRENCY degiskeninin %99u currency1, bu sebeple ayirt ediciligi olmayacagini dusundugumuz icin sildik  
    bureau_and_bb.drop(["SK_ID_BUREAU","CREDIT_CURRENCY"], inplace = True, axis = 1)


    #NEW FEATURES

    #ortalama kac aylık kredi aldıgını gösteren yeni degisken
    bureau_and_bb["NEW_MONTHS_CREDIT"]= round((bureau_and_bb.DAYS_CREDIT_ENDDATE - bureau_and_bb.DAYS_CREDIT)/30)

    agg_list = {
          "SK_ID_CURR":["count"],
          "DAYS_CREDIT":["min","max"],
          "CREDIT_DAY_OVERDUE":["sum","mean","max"],     
          "DAYS_CREDIT_ENDDATE":["max","min"],
          "DAYS_ENDDATE_FACT":["max","min"],
          "AMT_CREDIT_MAX_OVERDUE":["mean","max","min"],
          "CNT_CREDIT_PROLONG":["sum","mean","max","min"],
          "AMT_CREDIT_SUM":["mean","max","min"],            
          "AMT_CREDIT_SUM_DEBT":["sum","mean","max"],
          "AMT_CREDIT_SUM_LIMIT":["sum","mean","max"],
          'AMT_CREDIT_SUM_OVERDUE':["sum","mean","max"], 
          'DAYS_CREDIT_UPDATE':["max","min"],
          'AMT_ANNUITY':["sum","mean"],
          'MONTHS_BALANCE_COUNT':["sum"], 
          'STATUS_0_SUM':["sum"],         
          'STATUS_0_MEAN':["mean"], 
          'STATUS_C_SUM':["sum"], 
          'STATUS_C_MEAN':["mean"],
          'CREDIT_ACTIVE_Active':["sum","mean"], 
          'CREDIT_ACTIVE_Closed':["sum","mean"], 
          'CREDIT_TYPE_Rare':["sum","mean"],      
          'CREDIT_TYPE_Consumer credit':["sum","mean"], 
          'CREDIT_TYPE_Credit card':["sum","mean"],
          "NEW_MONTHS_CREDIT":["count","sum","mean","max","min"]}


    # bureau_bb_agg tablosuna aggreagation islemlerinin uygulanamasi  
    bureau_and_bb_agg = bureau_and_bb.groupby("SK_ID_CURR").agg(agg_list).reset_index()


    # Degisken isimlerinin yeniden adlandirilmasi 
    bureau_and_bb_agg.columns = pd.Index(["BB_" + col[0] + "_" + col[1].upper() for col in bureau_and_bb_agg.columns.tolist()])

    # kisinin aldıgı en yuksek ve en dusuk kredinin farkını gösteren yeni degisken
    bureau_and_bb_agg["BB_NEW_AMT_CREDIT_SUM_RANGE"] = bureau_and_bb_agg["BB_AMT_CREDIT_SUM_MAX"] - bureau_and_bb_agg["BB_AMT_CREDIT_SUM_MIN"]

    # ortalama kac ayda bir kredi cektigini ifade eden  yeni degisken
    bureau_and_bb_agg["BB_NEW_DAYS_CREDIT_RANGE"]= round((bureau_and_bb_agg["BB_DAYS_CREDIT_MAX"] - bureau_and_bb_agg["BB_DAYS_CREDIT_MIN"])/(30 * bureau_and_bb_agg["BB_SK_ID_CURR_COUNT"]))


    # Bureau: Active credits - using only numerical aggregations
    agg_list = {
            'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
            'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
            'DAYS_CREDIT_UPDATE': ['mean'],
            'CREDIT_DAY_OVERDUE': ['max', 'mean'],
            'AMT_CREDIT_MAX_OVERDUE': ['mean'],
            'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
            'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
            'AMT_CREDIT_SUM_OVERDUE': ['mean'],
            'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
            'AMT_ANNUITY': ['max', 'mean'],
            'CNT_CREDIT_PROLONG': ['sum']
        }


    active = bureau_and_bb[bureau_and_bb['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(agg_list)
    active_agg.columns = pd.Index(['BB_NEW_ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    bureau_and_bb_agg.rename(columns = {'BB_SK_ID_CURR_': 'SK_ID_CURR'}, inplace = True)
    bureau_and_bb_agg = bureau_and_bb_agg.join(active_agg, how='left', on='SK_ID_CURR')

    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau_and_bb[bureau_and_bb['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(agg_list)
    closed_agg.columns = pd.Index(['BB_NEW_CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    bureau_and_bb_agg = bureau_and_bb_agg.join(closed_agg, how='left', on='SK_ID_CURR')
    
    return bureau_and_bb_agg

In [8]:
def installments_payments():

    #Read the installments_payments.csv
    ins = pd.read_csv('./home-credit-default-risk/installments_payments.csv')

    ins['NEW_DAYS_PAID_EARLIER'] = ins['DAYS_INSTALMENT']-ins['DAYS_ENTRY_PAYMENT']

    # Her bir taksit ödemesinin gec olup olmama durumu 1: gec ödedi 0: erken ödemeyi temsil eder
    ins['NEW_NUM_PAID_LATER'] = ins['NEW_DAYS_PAID_EARLIER'].map(lambda x: 1 if x<0 else 0)

    # Agrregation ve degisken tekillestirme
    agg_list = {'NUM_INSTALMENT_VERSION':['nunique'],
               'NUM_INSTALMENT_NUMBER':'max',
               'DAYS_INSTALMENT':['min','max'],
               'DAYS_ENTRY_PAYMENT':['min','max'],
               'AMT_INSTALMENT':['min','max','sum','mean'],
               'AMT_PAYMENT':['min','max','sum','mean'],
               'NEW_DAYS_PAID_EARLIER':'mean',
               'NEW_NUM_PAID_LATER':'sum'}


    ins_agg = ins.groupby('SK_ID_PREV').agg(agg_list)


    # Multi index problemi cözümü
    ins_agg.columns = pd.Index(["INS_" + e[0] + '_' + e[1].upper() for e in ins_agg.columns.tolist()])

    # drop variables 
    ins_agg.drop(['INS_DAYS_INSTALMENT_MIN',
                   'INS_DAYS_INSTALMENT_MAX',
                   'INS_DAYS_ENTRY_PAYMENT_MIN',
                   'INS_DAYS_ENTRY_PAYMENT_MAX'],axis=1,inplace=True)

    # Kredi ödeme yüzdesi ve toplam kalan borc
    ins_agg['INS_NEW_PAYMENT_PERC'] = ins_agg['INS_AMT_PAYMENT_SUM'] / ins_agg['INS_AMT_INSTALMENT_SUM']
    ins_agg['INS_NEW_PAYMENT_DIFF'] = ins_agg['INS_AMT_INSTALMENT_SUM'] - ins_agg['INS_AMT_PAYMENT_SUM']
    
    agg_list_previous_application = {}
    
    for col in ins_agg.columns:
        agg_list_previous_application[col] = ['mean',"min","max","sum"]
    
    ins_agg.reset_index(inplace = True) 
    
    return agg_list_previous_application, ins_agg

In [9]:
def pos_cash_balance(agg_list_previous_application):

    pos = pd.read_csv('./home-credit-default-risk/POS_CASH_balance.csv')
    # Kategorik Degiskenimizi Dummy Degiskenine Dönüstürme
    pos = pd.get_dummies(pos, columns=['NAME_CONTRACT_STATUS'], dummy_na = True)
    # Aggregation Islemi - Tekillestirme
    agg_list = {'MONTHS_BALANCE':['min','max'],
                                            'CNT_INSTALMENT':['min','max'],
                                            'CNT_INSTALMENT_FUTURE':['min','max'],
                                            'SK_DPD':['max','mean'],
                                            'SK_DPD_DEF':['max','mean'],
                                            'NAME_CONTRACT_STATUS_Active':'sum',
                                            'NAME_CONTRACT_STATUS_Amortized debt':'sum',
                                            'NAME_CONTRACT_STATUS_Approved':'sum',
                                            'NAME_CONTRACT_STATUS_Canceled':'sum',
                                            'NAME_CONTRACT_STATUS_Completed':'sum',
                                            'NAME_CONTRACT_STATUS_Demand':'sum',
                                            'NAME_CONTRACT_STATUS_Returned to the store':'sum',
                                            'NAME_CONTRACT_STATUS_Signed':'sum',
                                            'NAME_CONTRACT_STATUS_XNA':'sum',
                                            'NAME_CONTRACT_STATUS_nan':'sum'
                                          }

    pos_agg = pos.groupby('SK_ID_PREV').agg(agg_list)

    # Multilayer index'i tek boyutlu index'e dönüstürme
    pos_agg.columns= pd.Index(["POS_" + e[0] + '_' + e[1].upper() for e in pos_agg.columns.tolist()])

    # SK_DPD kac kredide 0 olma durumu (SK_DPD MAX alacagiz 0 durumunu veriyor) 
    # SK_DPD_DEF (SK_DPD_DEF_MAX sifir olma durumunu veriyor)
    # CNT_INSTALMENT_FUTURE_MIN==0 oldugunda NAME_CONTRACT_STATUS_Completed_SUM==0 olma durumu 

    pos_agg['POS_NEW_IS_CREDIT_NOT_COMPLETED_ON_TIME']= (pos_agg['POS_CNT_INSTALMENT_FUTURE_MIN']==0) & (pos_agg['POS_NAME_CONTRACT_STATUS_Completed_SUM']==0)


    # 1:kredi zamaninda kapanmamis 0:kredi zamaninda kapanmis

    pos_agg['POS_NEW_IS_CREDIT_NOT_COMPLETED_ON_TIME']=pos_agg['POS_NEW_IS_CREDIT_NOT_COMPLETED_ON_TIME'].astype(int)

    pos_agg.drop(['POS_NAME_CONTRACT_STATUS_Approved_SUM',
                   'POS_NAME_CONTRACT_STATUS_Amortized debt_SUM',
                   'POS_NAME_CONTRACT_STATUS_Canceled_SUM',
                   'POS_NAME_CONTRACT_STATUS_Returned to the store_SUM',
                   'POS_NAME_CONTRACT_STATUS_Signed_SUM',
                   'POS_NAME_CONTRACT_STATUS_XNA_SUM',
                   'POS_NAME_CONTRACT_STATUS_nan_SUM'],axis=1,inplace=True)

    for col in pos_agg.columns:
        agg_list_previous_application[col] = ['mean',"min","max","sum"]

    pos_agg.reset_index(inplace = True)     
    
    return agg_list_previous_application, pos_agg

In [10]:
def credit_card_balance():

    CCB = pd.read_csv('./home-credit-default-risk/credit_card_balance.csv')

    CCB = pd.get_dummies(CCB, columns= ['NAME_CONTRACT_STATUS'] )  # artik tumu sayisal 

    dropthis = ['NAME_CONTRACT_STATUS_Approved', 'NAME_CONTRACT_STATUS_Demand',
           'NAME_CONTRACT_STATUS_Refused', 'NAME_CONTRACT_STATUS_Sent proposal',
           'NAME_CONTRACT_STATUS_Signed' ]

    CCB = CCB.drop(dropthis, axis=1)

    grp = CCB.groupby(by = ['SK_ID_CURR'])['SK_ID_PREV'].nunique().reset_index().rename(index = str, columns = {'SK_ID_PREV': 'NUMBER_OF_LOANS_PER_CUSTOMER'})
    CCB = CCB.merge(grp, on = ['SK_ID_CURR'], how = 'left')

    grp = CCB.groupby(by = ['SK_ID_CURR', 'SK_ID_PREV'])['CNT_INSTALMENT_MATURE_CUM'].max().reset_index().rename(index = str, columns = {'CNT_INSTALMENT_MATURE_CUM': 'NUMBER_OF_INSTALMENTS'})
    grp1 = grp.groupby(by = ['SK_ID_CURR'])['NUMBER_OF_INSTALMENTS'].sum().reset_index().rename(index = str, columns = {'NUMBER_OF_INSTALMENTS': 'TOTAL_INSTALMENTS_OF_ALL_LOANS'})
    CCB = CCB.merge(grp1, on = ['SK_ID_CURR'], how = 'left')

    CCB['INSTALLMENTS_PER_LOAN'] = (CCB['TOTAL_INSTALMENTS_OF_ALL_LOANS']/CCB['NUMBER_OF_LOANS_PER_CUSTOMER']).astype('uint32')


    # Bu fonksiyon, kac defa odemelerin geciktigini hesaplar   # Function to calculate number of times Days Past Due occurred 
    def geciken_gun_hesapla(DPD):

        # DPD ile beklenen bir seri: SK_DPD degiskeninin her bir prev_app daki gecmis kredi icin olan degerleri  # DPD is a series of values of SK_DPD for each of the groupby combination 
        # We convert it to a list to get the number of SK_DPD values NOT EQUALS ZERO
        x = DPD.tolist()
        c = 0
        for i,j in enumerate(x):
            if j != 0:
                c += 1  
        return c 

    grp = CCB.groupby(by = ['SK_ID_CURR', 'SK_ID_PREV']).apply(lambda x: geciken_gun_hesapla(x.SK_DPD)).reset_index().rename(index = str, columns = {0: 'NUMBER_OF_DPD'})
    grp1 = grp.groupby(by = ['SK_ID_CURR'])['NUMBER_OF_DPD'].mean().reset_index().rename(index = str, columns = {'NUMBER_OF_DPD' : 'DPD_COUNT'})

    CCB = CCB.merge(grp1, on = ['SK_ID_CURR'], how = 'left')


    def f(min_pay, total_pay):

        M = min_pay.tolist()
        T = total_pay.tolist()
        P = len(M)        # P: taksit sayisi
        c = 0 
        # Find the count of transactions when Payment made is less than Minimum Payment 
        for i in range(len(M)):
            if T[i] < M[i]:
                c += 1  
        return (100*c)/P

    grp = CCB.groupby(by = ['SK_ID_CURR']).apply(lambda x: f(x.AMT_INST_MIN_REGULARITY, x.AMT_PAYMENT_CURRENT)).reset_index().rename(index = str, columns = { 0 : 'PERCENTAGE_MIN_MISSED_PAYMENTS'})
    CCB = CCB.merge(grp, on = ['SK_ID_CURR'], how = 'left')

    grp = CCB.groupby(by = ['SK_ID_CURR'])['AMT_DRAWINGS_ATM_CURRENT'].sum().reset_index().rename(index = str, columns = {'AMT_DRAWINGS_ATM_CURRENT' : 'DRAWINGS_ATM'})
    CCB = CCB.merge(grp, on = ['SK_ID_CURR'], how = 'left')


    grp = CCB.groupby(by = ['SK_ID_CURR'])['AMT_DRAWINGS_CURRENT'].sum().reset_index().rename(index = str, columns = {'AMT_DRAWINGS_CURRENT' : 'DRAWINGS_TOTAL'})
    CCB = CCB.merge(grp, on = ['SK_ID_CURR'], how = 'left')


    CCB['CASH_CARD_RATIO1'] = (CCB['DRAWINGS_ATM']/CCB['DRAWINGS_TOTAL'])*100  # ATM den cektigi nakit / toplam cektigi
    del CCB['DRAWINGS_ATM']
    del CCB['DRAWINGS_TOTAL']

    grp = CCB.groupby(by = ['SK_ID_CURR'])['CASH_CARD_RATIO1'].mean().reset_index().rename(index = str, columns ={ 'CASH_CARD_RATIO1' : 'CASH_CARD_RATIO'})
    CCB = CCB.merge(grp, on = ['SK_ID_CURR'], how = 'left')


    grp = CCB.groupby(by = ['SK_ID_CURR'])['AMT_DRAWINGS_CURRENT'].sum().reset_index().rename(index = str, columns = {'AMT_DRAWINGS_CURRENT' : 'TOTAL_DRAWINGS'})
    CCB = CCB.merge(grp, on = ['SK_ID_CURR'], how = 'left')


    grp = CCB.groupby(by = ['SK_ID_CURR'])['CNT_DRAWINGS_CURRENT'].sum().reset_index().rename(index = str, columns = {'CNT_DRAWINGS_CURRENT' : 'NUMBER_OF_DRAWINGS'})
    CCB = CCB.merge(grp, on = ['SK_ID_CURR'], how = 'left')


    CCB['DRAWINGS_RATIO1'] = (CCB['TOTAL_DRAWINGS']/CCB['NUMBER_OF_DRAWINGS'])*100     # yuzdelik degil, genisleme yapmis
    del CCB['TOTAL_DRAWINGS']
    del CCB['NUMBER_OF_DRAWINGS']


    grp = CCB.groupby(by = ['SK_ID_CURR'])['DRAWINGS_RATIO1'].mean().reset_index().rename(index = str, columns ={ 'DRAWINGS_RATIO1' : 'DRAWINGS_RATIO'})
    CCB = CCB.merge(grp, on = ['SK_ID_CURR'], how = 'left')

    del CCB['DRAWINGS_RATIO1']

    CCB['CC_COUNT'] = CCB.groupby('SK_ID_CURR').size()

    CCB_agg = CCB.groupby('SK_ID_CURR').agg({
        'MONTHS_BALANCE':["sum","mean"], 
        'AMT_BALANCE':["sum","mean","min","max"],
        'AMT_CREDIT_LIMIT_ACTUAL':["sum","mean"], 

        'AMT_DRAWINGS_ATM_CURRENT':["sum","mean","min","max"],
        'AMT_DRAWINGS_CURRENT':["sum","mean","min","max"], 
        'AMT_DRAWINGS_OTHER_CURRENT':["sum","mean","min","max"],
        'AMT_DRAWINGS_POS_CURRENT':["sum","mean","min","max"], 
        'AMT_INST_MIN_REGULARITY':["sum","mean","min","max"],
        'AMT_PAYMENT_CURRENT':["sum","mean","min","max"], 
        'AMT_PAYMENT_TOTAL_CURRENT':["sum","mean","min","max"],
        'AMT_RECEIVABLE_PRINCIPAL':["sum","mean","min","max"], 
        'AMT_RECIVABLE':["sum","mean","min","max"], 
        'AMT_TOTAL_RECEIVABLE':["sum","mean","min","max"],

        'CNT_DRAWINGS_ATM_CURRENT':["sum","mean"], 
        'CNT_DRAWINGS_CURRENT':["sum","mean","max"],
        'CNT_DRAWINGS_OTHER_CURRENT':["mean","max"], 
        'CNT_DRAWINGS_POS_CURRENT':["sum","mean","max"],
        'CNT_INSTALMENT_MATURE_CUM':["sum","mean","max","min"],    
        'SK_DPD':["sum","mean","max"], 
        'SK_DPD_DEF':["sum","mean","max"],

        'NAME_CONTRACT_STATUS_Active':["sum","mean","min","max"], 
        'INSTALLMENTS_PER_LOAN':["sum","mean","min","max"],

        'NUMBER_OF_LOANS_PER_CUSTOMER':["mean"], 
        'DPD_COUNT':["mean"],
        'PERCENTAGE_MIN_MISSED_PAYMENTS':["mean"], 
        'CASH_CARD_RATIO':["mean"], 
        'DRAWINGS_RATIO':["mean"]})


    CCB_agg.columns = pd.Index(['CCB_' + e[0] + "_" + e[1].upper() for e in CCB_agg.columns.tolist()])

    CCB_agg.reset_index(inplace = True)
    
    return CCB_agg

In [11]:
def previous_application(agg_list_previous_application):


    df_prev = pd.read_csv('./home-credit-default-risk/previous_application.csv')

    # "WEEKDAY_APPR_PROCESS_START"  değişkeninin  WEEK_DAY ve WEEKEND olarak iki kategoriye ayrılması

    df_prev["WEEKDAY_APPR_PROCESS_START"] = df_prev["WEEKDAY_APPR_PROCESS_START"].replace(['MONDAY','TUESDAY', 'WEDNESDAY','THURSDAY','FRIDAY'], 'WEEK_DAY')
    df_prev["WEEKDAY_APPR_PROCESS_START"] = df_prev["WEEKDAY_APPR_PROCESS_START"].replace(['SATURDAY', 'SUNDAY'], 'WEEKEND')

    # "HOUR_APPR_PROCESS_START"  değişkeninin working_hours ve off_hours olarak iki kategoriye ayrılması
    a = [8,9,10,11,12,13,14,15,16,17]
    df_prev["HOUR_APPR_PROCESS_START"] = df_prev["HOUR_APPR_PROCESS_START"].replace(a, 'working_hours')

    b = [18,19,20,21,22,23,0,1,2,3,4,5,6,7]
    df_prev["HOUR_APPR_PROCESS_START"] = df_prev["HOUR_APPR_PROCESS_START"].replace(b, 'off_hours')


    # DAYS_DECISION değeri 1 yıldan küçük olanlara 1, büyük olanlara 0 değeri verildi.
    df_prev["DAYS_DECISION"] = [1 if abs(i/(12*30)) <=1 else 0 for i in df_prev.DAYS_DECISION]

    # "NAME_TYPE_SUITE"  değişkeninin alone ve not_alone olarak iki kategoriye ayrılması

    df_prev["NAME_TYPE_SUITE"] = df_prev["NAME_TYPE_SUITE"].replace('Unaccompanied', 'alone')

    b = ['Family', 'Spouse, partner', 'Children', 'Other_B', 'Other_A', 'Group of people']
    df_prev["NAME_TYPE_SUITE"] = df_prev["NAME_TYPE_SUITE"].replace(b, 'not_alone')



    # "NAME_GOODS_CATEGORY"  değişkenindeki bu değerler others olarak kategorize edilecek
    a = ['Auto Accessories', 'Jewelry', 'Homewares', 'Medical Supplies', 'Vehicles', 'Sport and Leisure', 
         'Gardening', 'Other', 'Office Appliances', 'Tourism', 'Medicine', 'Direct Sales', 'Fitness', 'Additional Service', 
         'Education', 'Weapon', 'Insurance', 'House Construction', 'Animals'] 
    df_prev["NAME_GOODS_CATEGORY"] = df_prev["NAME_GOODS_CATEGORY"].replace(a, 'others')

    # "NAME_SELLER_INDUSTRY"  değişkenindeki bu değerler others olarak kategorize edilecek
    a = ['Auto technology', 'Jewelry', 'MLM partners', 'Tourism'] 
    df_prev["NAME_SELLER_INDUSTRY"] = df_prev["NAME_SELLER_INDUSTRY"].replace(a, 'others')
    # İstenilen krecinin verilen krediye oranı içeren değişkeni türetir
    df_prev["LOAN_RATE"] = df_prev.AMT_APPLICATION/df_prev.AMT_CREDIT

    #YENI DEGISKENLER

    # İstenilen krecinin verilen krediye oranı içeren değişkeni türetir
    df_prev["NEW_LOAN_RATE"] = df_prev.AMT_APPLICATION/df_prev.AMT_CREDIT

    # Ödeme gününü geciktirmiş mi bunu gösteren churn_prev değişkeni türetilir.
    # 1= geciktirmiş, 0 = geciktirmemiş, NaN = boş değer
    k = df_prev.DAYS_LAST_DUE_1ST_VERSION - df_prev.DAYS_LAST_DUE
    df_prev["NEW_CHURN_PREV"] = [1 if i >= 0 else (0 if i < 0  else "NaN") for i in k]


    # NFLAG_INSURED_ON_APPROVAL değişkeni yerine kullanılmak izere NEW_INSURANCE değişkeni tanımlandı.
    df_prev[(df_prev['AMT_CREDIT'] == 0) | (df_prev['AMT_GOODS_PRICE'] == 0)]['NEW_INSURANCE'] = np.nan
    df_prev['sigorta_miktari'] = df_prev['AMT_CREDIT'] - df_prev['AMT_GOODS_PRICE']
    df_prev["NEW_INSURANCE"] = df_prev['sigorta_miktari'].apply(lambda x: 1 if x > 0 else (0 if x <= 0 else np.nan))
    df_prev.drop('sigorta_miktari', axis=1, inplace=True)

    # INTEREST_RATE değişkenini oluşturur.
    #df_prev['INTEREST_RATE'] = (df_prev.AMT_ANNUITY*df_prev.CNT_PAYMENT/df_prev.AMT_CREDIT)**(12/df_prev.CNT_PAYMENT)-1
    #df_prev[df_prev['INTEREST_RATE']==-1]=np.nan


    drop_list = ['AMT_DOWN_PAYMENT', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'PRODUCT_COMBINATION', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE',
                'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE','DAYS_TERMINATION','NFLAG_INSURED_ON_APPROVAL']
    df_prev.drop(drop_list, axis = 1, inplace = True)

    # Previous tablosundaki kategorik değişkenlerin isimlerini tutar.
    category_columns=[]
    for i in df_prev.columns:
        if df_prev[i].dtypes == "O":
            category_columns.append(i)

    df_prev = pd.get_dummies(df_prev, columns = category_columns )

    prev_agg_list = {"SK_ID_CURR":["count"], 
                "AMT_ANNUITY":["max"],
                "AMT_APPLICATION":["min","mean","max"],
                "AMT_CREDIT":["max"], 
                "AMT_GOODS_PRICE":["sum", "mean"],
                "NFLAG_LAST_APPL_IN_DAY":["sum","mean"], 
                "RATE_DOWN_PAYMENT":["sum", "mean"],
                "RATE_INTEREST_PRIMARY":["sum", "mean"],
                "RATE_INTEREST_PRIVILEGED":["sum", "mean"],
                "DAYS_DECISION":["sum"],
                "NEW_LOAN_RATE":["sum", "mean", "min", "max"],
                "NEW_INSURANCE":["sum", "mean"],
                #"INTEREST_RATE":["sum", "mean", "min", "max"],
                "NAME_CONTRACT_TYPE_Cash loans":["sum", "mean"],
                "NAME_CONTRACT_TYPE_Consumer loans":["sum", "mean"],
                "NAME_CONTRACT_TYPE_Revolving loans":["sum", "mean"],
                "NAME_CONTRACT_TYPE_XNA":["sum", "mean"],
                "WEEKDAY_APPR_PROCESS_START_WEEKEND":["sum", "mean"],
                "WEEKDAY_APPR_PROCESS_START_WEEK_DAY":["sum", "mean"],
                "HOUR_APPR_PROCESS_START_off_hours":["sum", "mean"],
                "HOUR_APPR_PROCESS_START_working_hours":["sum", "mean"],
                "FLAG_LAST_APPL_PER_CONTRACT_N":["sum", "mean"],
                "FLAG_LAST_APPL_PER_CONTRACT_Y":["sum", "mean"],
                "NAME_CASH_LOAN_PURPOSE_Building a house or an annex":["sum", "mean"],
                "NAME_CASH_LOAN_PURPOSE_Business development":["sum", "mean"],
                "NAME_CASH_LOAN_PURPOSE_Buying a garage":["sum", "mean"],
                "NAME_CASH_LOAN_PURPOSE_Buying a holiday home / land":["sum", "mean"],
                "NAME_CASH_LOAN_PURPOSE_Buying a home":["sum", "mean"],
                "NAME_CASH_LOAN_PURPOSE_Buying a new car":["sum", "mean"],
                "NAME_CASH_LOAN_PURPOSE_Buying a used car":["sum", "mean"],
                "NAME_CASH_LOAN_PURPOSE_Car repairs":["sum", "mean"],
                "NAME_CASH_LOAN_PURPOSE_Education":["sum", "mean"],
                "NAME_CASH_LOAN_PURPOSE_Everyday expenses":["sum", "mean"],
                "NAME_CASH_LOAN_PURPOSE_Furniture":["sum", "mean"],
                "NAME_CASH_LOAN_PURPOSE_Gasification / water supply":["sum", "mean"],
                "NAME_CASH_LOAN_PURPOSE_Hobby":["sum", "mean"],
                "NAME_CASH_LOAN_PURPOSE_Journey":["sum", "mean"],
                "NAME_CASH_LOAN_PURPOSE_Medicine":["sum", "mean"],
                "NAME_CASH_LOAN_PURPOSE_Money for a third person":["sum", "mean"],
                "NAME_CASH_LOAN_PURPOSE_Other":["sum", "mean"],
                "NAME_CASH_LOAN_PURPOSE_Payments on other loans":["sum", "mean"],
                "NAME_CASH_LOAN_PURPOSE_Purchase of electronic equipment":["sum", "mean"],
                "NAME_CASH_LOAN_PURPOSE_Refusal to name the goal":["sum", "mean"],
                "NAME_CASH_LOAN_PURPOSE_Repairs":["sum", "mean"],
                "NAME_CASH_LOAN_PURPOSE_Urgent needs":["sum", "mean"],
                "NAME_CASH_LOAN_PURPOSE_Wedding / gift / holiday":["sum", "mean"],
                "NAME_CASH_LOAN_PURPOSE_XAP":["sum", "mean"],
                "NAME_CASH_LOAN_PURPOSE_XNA":["sum", "mean"],
                "NAME_CONTRACT_STATUS_Approved":["sum", "mean"],
                "NAME_CONTRACT_STATUS_Canceled":["sum", "mean"],
                "NAME_CONTRACT_STATUS_Refused":["sum", "mean"],
                "NAME_CONTRACT_STATUS_Unused offer":["sum", "mean"],
                "NAME_PAYMENT_TYPE_Cash through the bank":["sum", "mean"],
                "NAME_PAYMENT_TYPE_Cashless from the account of the employer":["sum", "mean"],
                "NAME_PAYMENT_TYPE_Non-cash from your account":["sum", "mean"],
                "NAME_PAYMENT_TYPE_XNA":["sum", "mean"],
                "CODE_REJECT_REASON_CLIENT":["sum", "mean"],
                "CODE_REJECT_REASON_HC":["sum", "mean"],
                "CODE_REJECT_REASON_LIMIT":["sum", "mean"],
                "CODE_REJECT_REASON_SCO":["sum", "mean"],
                "CODE_REJECT_REASON_SCOFR":["sum", "mean"],
                "CODE_REJECT_REASON_SYSTEM":["sum", "mean"],
                "CODE_REJECT_REASON_VERIF":["sum", "mean"],
                "CODE_REJECT_REASON_XAP":["sum", "mean"],
                "CODE_REJECT_REASON_XNA":["sum", "mean"],
                "NAME_TYPE_SUITE_alone":["sum", "mean"],
                "NAME_TYPE_SUITE_not_alone":["sum", "mean"],
                "NAME_CLIENT_TYPE_New":["sum", "mean"],
                "NAME_CLIENT_TYPE_Refreshed":["sum", "mean"],
                "NAME_CLIENT_TYPE_Repeater":["sum", "mean"],
                "NAME_CLIENT_TYPE_XNA":["sum", "mean"],
                "NAME_GOODS_CATEGORY_Audio/Video":["sum", "mean"],
                "NAME_GOODS_CATEGORY_Clothing and Accessories":["sum", "mean"],
                "NAME_GOODS_CATEGORY_Computers":["sum", "mean"],
                "NAME_GOODS_CATEGORY_Construction Materials":["sum", "mean"],
                "NAME_GOODS_CATEGORY_Consumer Electronics":["sum", "mean"],
                "NAME_GOODS_CATEGORY_Furniture":["sum", "mean"],
                "NAME_GOODS_CATEGORY_Mobile":["sum", "mean"],
                "NAME_GOODS_CATEGORY_Photo / Cinema Equipment":["sum", "mean"],
                "NAME_GOODS_CATEGORY_XNA":["sum", "mean"],
                "NAME_GOODS_CATEGORY_others":["sum", "mean"],
                "NAME_PORTFOLIO_Cards":["sum", "mean"],
                "NAME_PORTFOLIO_Cars":["sum", "mean"],
                "NAME_PORTFOLIO_Cash":["sum", "mean"],
                "NAME_PORTFOLIO_POS":["sum", "mean"],
                "NAME_PORTFOLIO_XNA":["sum", "mean"],
                "NAME_PRODUCT_TYPE_XNA":["sum", "mean"],
                "NAME_PRODUCT_TYPE_walk-in":["sum", "mean"],
                "NAME_PRODUCT_TYPE_x-sell":["sum", "mean"],
                "CHANNEL_TYPE_AP+ (Cash loan)":["sum", "mean"],
                "CHANNEL_TYPE_Car dealer":["sum", "mean"],
                "CHANNEL_TYPE_Channel of corporate sales":["sum", "mean"],
                "CHANNEL_TYPE_Contact center":["sum", "mean"],
                "CHANNEL_TYPE_Country-wide":["sum", "mean"],
                "CHANNEL_TYPE_Credit and cash offices":["sum", "mean"],
                "CHANNEL_TYPE_Regional / Local":["sum", "mean"],
                "CHANNEL_TYPE_Stone":["sum", "mean"],
                "NAME_SELLER_INDUSTRY_Clothing":["sum", "mean"],
                "NAME_SELLER_INDUSTRY_Connectivity":["sum", "mean"],
                "NAME_SELLER_INDUSTRY_Construction":["sum", "mean"],
                "NAME_SELLER_INDUSTRY_Consumer electronics":["sum", "mean"],
                "NAME_SELLER_INDUSTRY_Furniture":["sum", "mean"],
                "NAME_SELLER_INDUSTRY_Industry":["sum", "mean"],
                "NAME_SELLER_INDUSTRY_XNA":["sum", "mean"],
                "NAME_SELLER_INDUSTRY_others":["sum", "mean"],
                "NAME_YIELD_GROUP_XNA":["sum", "mean"],
                "NAME_YIELD_GROUP_high":["sum", "mean"],
                "NAME_YIELD_GROUP_low_action":["sum", "mean"],
                "NAME_YIELD_GROUP_low_normal":["sum", "mean"],
                "NAME_YIELD_GROUP_middle":["sum", "mean"],
                "NEW_CHURN_PREV_0":["sum", "mean"],
                "NEW_CHURN_PREV_1":["sum", "mean"],
                "NEW_CHURN_PREV_NaN":["sum", "mean"]}

    prev_agg_list.update(agg_list_previous_application)
    
    
    return prev_agg_list, df_prev

In [12]:
def pre_processing_and_combine():

    
    with timer("Process application train"):
        df = application_train()
        print("application train & test shape:", df.shape)
        
    
    with timer("Bureau and Bureau Balance"):
        bureau_and_bb_agg = bureau_bb()
        print("Bureau and Bureau Balance:", bureau_and_bb_agg.shape)
        
    with timer("Installment Payments"):
        agg_list_previous_application, ins_agg = installments_payments()
        print("Installment Payments:", ins_agg.shape)    
    
    with timer("Pos Cash Balance"):
        agg_list_previous_application, pos_agg = pos_cash_balance(agg_list_previous_application)
        print("Pos Cash Balance:", pos_agg.shape)  
        
    
    with timer("Credit Card Balance"):
        CCB_agg = credit_card_balance()
        print("Credit Card Balance:", CCB_agg.shape) 
    
    with timer("previous_application"):
        prev_agg_list, df_prev = previous_application(agg_list_previous_application)
        print("previous_application:", df_prev.shape) 
        
        
    with timer("All tables are combining"):
        df_prev_ins = df_prev.merge(ins_agg, how = 'left', on = 'SK_ID_PREV')
        df_prev_ins_pos = df_prev_ins.merge(pos_agg, how = 'left', on = 'SK_ID_PREV')
        df_prev_ins_pos_agg = df_prev_ins_pos.groupby("SK_ID_CURR").agg(prev_agg_list).reset_index()
        df_prev_ins_pos_agg.columns = pd.Index(["PREV_" + col[0] + "_" + col[1].upper() for col in df_prev_ins_pos_agg.columns.tolist()])
        df_prev_ins_pos_agg.rename(columns={"PREV_SK_ID_CURR_":"SK_ID_CURR"}, inplace = True)
        #prev_son ile ana tablo
        df_prev_others = df.merge(df_prev_ins_pos_agg, how = 'left',on = 'SK_ID_CURR')
    
        #credit_card_balance
        df_prev_ins_pos_ccb = df_prev_others.merge(CCB_agg, how = 'left',on = 'SK_ID_CURR')
    
        #bureau_balance
        all_data = df_prev_ins_pos_ccb.merge(bureau_and_bb_agg, how = 'left',on = 'SK_ID_CURR')
        
        print("all_data process:", all_data.shape) 

    
    
    return all_data
    

In [13]:
with timer("Preprocessing Time"):
    df = pre_processing_and_combine()

df.head()

application train & test shape: (356251, 260)
Process application train - done in 4s
Bureau and Bureau Balance: (305811, 102)
Bureau and Bureau Balance - done in 15s
Installment Payments: (997752, 15)
Installment Payments - done in 23s
Pos Cash Balance: (936325, 15)
Pos Cash Balance - done in 10s
Credit Card Balance: (103558, 82)
Credit Card Balance - done in 34s
previous_application: (1670214, 114)
previous_application - done in 10s
all_data process: (356251, 777)
All tables are combining - done in 62s
Preprocessing Time - done in 157s


,SK_ID_CURR,TARGET,APP_CODE_GENDER,APP_FLAG_OWN_CAR,APP_FLAG_OWN_REALTY,APP_CNT_CHILDREN,APP_AMT_INCOME_TOTAL,APP_AMT_CREDIT,APP_AMT_ANNUITY,APP_AMT_GOODS_PRICE,...,BB_NEW_CLOSED_AMT_CREDIT_SUM_SUM,BB_NEW_CLOSED_AMT_CREDIT_SUM_DEBT_MAX,BB_NEW_CLOSED_AMT_CREDIT_SUM_DEBT_MEAN,BB_NEW_CLOSED_AMT_CREDIT_SUM_DEBT_SUM,BB_NEW_CLOSED_AMT_CREDIT_SUM_OVERDUE_MEAN,BB_NEW_CLOSED_AMT_CREDIT_SUM_LIMIT_MEAN,BB_NEW_CLOSED_AMT_CREDIT_SUM_LIMIT_SUM,BB_NEW_CLOSED_AMT_ANNUITY_MAX,BB_NEW_CLOSED_AMT_ANNUITY_MEAN,BB_NEW_CLOSED_CNT_CREDIT_PROLONG_SUM
0,100002,1.0,1,0,1,0,202500.0,406597.5,24700.5,351000.0,...,383067.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100003,0.0,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,...,207400.5,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0
2,100004,0.0,1,1,1,0,67500.0,135000.0,6750.0,135000.0,...,189037.8,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0
3,100006,0.0,0,0,1,0,135000.0,312682.5,29686.5,297000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0.0,1,0,1,0,121500.0,513000.0,21865.5,513000.0,...,146250.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0


In [14]:
df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in df.columns]

In [15]:
### infをnanへ置換
df = df.replace([np.inf, -np.inf], np.nan)
df.head()

,SK_ID_CURR,TARGET,APP_CODE_GENDER,APP_FLAG_OWN_CAR,APP_FLAG_OWN_REALTY,APP_CNT_CHILDREN,APP_AMT_INCOME_TOTAL,APP_AMT_CREDIT,APP_AMT_ANNUITY,APP_AMT_GOODS_PRICE,...,BB_NEW_CLOSED_AMT_CREDIT_SUM_SUM,BB_NEW_CLOSED_AMT_CREDIT_SUM_DEBT_MAX,BB_NEW_CLOSED_AMT_CREDIT_SUM_DEBT_MEAN,BB_NEW_CLOSED_AMT_CREDIT_SUM_DEBT_SUM,BB_NEW_CLOSED_AMT_CREDIT_SUM_OVERDUE_MEAN,BB_NEW_CLOSED_AMT_CREDIT_SUM_LIMIT_MEAN,BB_NEW_CLOSED_AMT_CREDIT_SUM_LIMIT_SUM,BB_NEW_CLOSED_AMT_ANNUITY_MAX,BB_NEW_CLOSED_AMT_ANNUITY_MEAN,BB_NEW_CLOSED_CNT_CREDIT_PROLONG_SUM
0,100002,1.0,1,0,1,0,202500.0,406597.5,24700.5,351000.0,...,383067.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100003,0.0,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,...,207400.5,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0
2,100004,0.0,1,1,1,0,67500.0,135000.0,6750.0,135000.0,...,189037.8,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0
3,100006,0.0,0,0,1,0,135000.0,312682.5,29686.5,297000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0.0,1,0,1,0,121500.0,513000.0,21865.5,513000.0,...,146250.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0


In [16]:
### 変数選択001
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 777 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 770 features are remained after removing non-informative features
107 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 663 features are remained after removing features not interesting for LightGBM classifier
---=> 70 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 733 features

---=> df final shape: (356251, 733)  <=--- 



,SK_ID_CURR,TARGET,APP_CODE_GENDER,APP_FLAG_OWN_CAR,APP_FLAG_OWN_REALTY,APP_CNT_CHILDREN,APP_AMT_INCOME_TOTAL,APP_AMT_CREDIT,APP_AMT_ANNUITY,APP_AMT_GOODS_PRICE,...,PREV_NEW_CHURN_PREV_0_SUM_high_risk,PREV_NEW_CHURN_PREV_0_SUM_low_risk,PREV_POS_NAME_CONTRACT_STATUS_Completed_SUM_MIN_high_risk,PREV_POS_NAME_CONTRACT_STATUS_Completed_SUM_MIN_low_risk,CCB_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CCB_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,BB_CNT_CREDIT_PROLONG_MAX_high_risk,BB_CNT_CREDIT_PROLONG_MAX_low_risk,BB_NEW_ACTIVE_CNT_CREDIT_PROLONG_SUM_high_risk,BB_NEW_ACTIVE_CNT_CREDIT_PROLONG_SUM_low_risk
0,100002,1.0,1,0,1,0,202500.0,406597.5,24700.5,351000.0,...,1,0,0,1,0,0,0,1,1,1
1,100003,0.0,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,...,1,0,0,1,0,0,0,1,1,1
2,100004,0.0,1,1,1,0,67500.0,135000.0,6750.0,135000.0,...,1,0,1,0,0,0,0,1,0,0
3,100006,0.0,0,0,1,0,135000.0,312682.5,29686.5,297000.0,...,0,1,0,1,0,0,0,0,0,0
4,100007,0.0,1,0,1,0,121500.0,513000.0,21865.5,513000.0,...,0,1,0,1,0,0,0,1,0,0


In [17]:
### 変数選択002
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 733 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 728 features are remained after removing non-informative features
59 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 669 features are remained after removing features not interesting for LightGBM classifier
---=> 33 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 702 features

---=> df final shape: (356251, 702)  <=--- 



,SK_ID_CURR,TARGET,APP_CODE_GENDER,APP_FLAG_OWN_CAR,APP_FLAG_OWN_REALTY,APP_CNT_CHILDREN,APP_AMT_INCOME_TOTAL,APP_AMT_CREDIT,APP_AMT_ANNUITY,APP_AMT_GOODS_PRICE,...,PREV_NAME_GOODS_CATEGORY_others_SUM_high_risk,PREV_NAME_GOODS_CATEGORY_others_SUM_low_risk,PREV_NAME_SELLER_INDUSTRY_Construction_SUM_high_risk,PREV_NAME_SELLER_INDUSTRY_Furniture_SUM_low_risk,PREV_NAME_SELLER_INDUSTRY_Industry_SUM_high_risk,CCB_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CCB_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,BB_CNT_CREDIT_PROLONG_MAX_high_risk,BB_CNT_CREDIT_PROLONG_MAX_low_risk,BB_NEW_ACTIVE_CNT_CREDIT_PROLONG_SUM_low_risk
0,100002,1.0,1,0,1,0,202500.0,406597.5,24700.5,351000.0,...,0,1,1,0,1,0,0,0,1,1
1,100003,0.0,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,...,1,0,1,1,1,0,0,0,1,1
2,100004,0.0,1,1,1,0,67500.0,135000.0,6750.0,135000.0,...,1,0,1,0,1,0,0,0,1,0
3,100006,0.0,0,0,1,0,135000.0,312682.5,29686.5,297000.0,...,1,0,0,0,1,0,0,0,0,0
4,100007,0.0,1,0,1,0,121500.0,513000.0,21865.5,513000.0,...,1,0,1,0,1,0,0,0,1,0


In [18]:
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
df.head()

,SK_ID_CURR,TARGET,APP_CODE_GENDER,APP_FLAG_OWN_CAR,APP_FLAG_OWN_REALTY,APP_CNT_CHILDREN,APP_AMT_INCOME_TOTAL,APP_AMT_CREDIT,APP_AMT_ANNUITY,APP_AMT_GOODS_PRICE,...,PREV_NAME_GOODS_CATEGORY_others_SUM_high_risk,PREV_NAME_GOODS_CATEGORY_others_SUM_low_risk,PREV_NAME_SELLER_INDUSTRY_Construction_SUM_high_risk,PREV_NAME_SELLER_INDUSTRY_Furniture_SUM_low_risk,PREV_NAME_SELLER_INDUSTRY_Industry_SUM_high_risk,CCB_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CCB_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,BB_CNT_CREDIT_PROLONG_MAX_high_risk,BB_CNT_CREDIT_PROLONG_MAX_low_risk,BB_NEW_ACTIVE_CNT_CREDIT_PROLONG_SUM_low_risk
0,100002,1.0,1,0,1,0,202500.0,406597.5,24700.5,351000.0,...,0,1,1,0,1,0,0,0,1,1
1,100003,0.0,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,...,1,0,1,1,1,0,0,0,1,1
2,100004,0.0,1,1,1,0,67500.0,135000.0,6750.0,135000.0,...,1,0,1,0,1,0,0,0,1,0
3,100006,0.0,0,0,1,0,135000.0,312682.5,29686.5,297000.0,...,1,0,0,0,1,0,0,0,0,0
4,100007,0.0,1,0,1,0,121500.0,513000.0,21865.5,513000.0,...,1,0,1,0,1,0,0,0,1,0


In [19]:
### split train, and test datasets
app_train = df[df['TARGET'].notnull()]
app_test = df[df['TARGET'].isnull()]

print(app_test.shape)
app_test.head()

(48744, 702)


,SK_ID_CURR,TARGET,APP_CODE_GENDER,APP_FLAG_OWN_CAR,APP_FLAG_OWN_REALTY,APP_CNT_CHILDREN,APP_AMT_INCOME_TOTAL,APP_AMT_CREDIT,APP_AMT_ANNUITY,APP_AMT_GOODS_PRICE,...,PREV_NAME_GOODS_CATEGORY_others_SUM_high_risk,PREV_NAME_GOODS_CATEGORY_others_SUM_low_risk,PREV_NAME_SELLER_INDUSTRY_Construction_SUM_high_risk,PREV_NAME_SELLER_INDUSTRY_Furniture_SUM_low_risk,PREV_NAME_SELLER_INDUSTRY_Industry_SUM_high_risk,CCB_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CCB_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,BB_CNT_CREDIT_PROLONG_MAX_high_risk,BB_CNT_CREDIT_PROLONG_MAX_low_risk,BB_NEW_ACTIVE_CNT_CREDIT_PROLONG_SUM_low_risk
307507,100001,NaN,0,0,1,0,135000.0,568800.0,20560.5,450000.0,...,1,0,1,0,1,0,0,0,1,1
307508,100005,NaN,1,0,1,0,99000.0,222768.0,17370.0,180000.0,...,1,0,1,0,1,0,0,0,1,1
307509,100013,NaN,1,1,1,0,202500.0,663264.0,69777.0,630000.0,...,1,0,1,0,1,1,0,0,1,0
307510,100028,NaN,0,0,1,2,315000.0,1575000.0,49018.5,1575000.0,...,1,0,1,0,1,1,0,0,1,1
307511,100038,NaN,1,1,0,1,180000.0,625500.0,32067.0,625500.0,...,1,0,1,0,1,0,0,0,0,0


In [20]:
train_setB1a = pd.read_csv('./home-credit-default-risk/exports/train_setB1a.csv')
test_setB1a =  pd.read_csv('./home-credit-default-risk/exports/train_setB1a.csv')
test_setB1a.head()

,SK_ID_CURR,setB1a
0,100002,0.283935
1,100003,0.023060
2,100004,0.032398
3,100006,0.028579
4,100007,0.023252


In [21]:
app_train = app_train.merge(train_setB1a, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_setB1a, how='left', on='SK_ID_CURR')
app_test.head()

,SK_ID_CURR,TARGET,APP_CODE_GENDER,APP_FLAG_OWN_CAR,APP_FLAG_OWN_REALTY,APP_CNT_CHILDREN,APP_AMT_INCOME_TOTAL,APP_AMT_CREDIT,APP_AMT_ANNUITY,APP_AMT_GOODS_PRICE,...,PREV_NAME_GOODS_CATEGORY_others_SUM_low_risk,PREV_NAME_SELLER_INDUSTRY_Construction_SUM_high_risk,PREV_NAME_SELLER_INDUSTRY_Furniture_SUM_low_risk,PREV_NAME_SELLER_INDUSTRY_Industry_SUM_high_risk,CCB_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CCB_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,BB_CNT_CREDIT_PROLONG_MAX_high_risk,BB_CNT_CREDIT_PROLONG_MAX_low_risk,BB_NEW_ACTIVE_CNT_CREDIT_PROLONG_SUM_low_risk,setB1a
0,100001,NaN,0,0,1,0,135000.0,568800.0,20560.5,450000.0,...,0,1,0,1,0,0,0,1,1,NaN
1,100005,NaN,1,0,1,0,99000.0,222768.0,17370.0,180000.0,...,0,1,0,1,0,0,0,1,1,NaN
2,100013,NaN,1,1,1,0,202500.0,663264.0,69777.0,630000.0,...,0,1,0,1,1,0,0,1,0,NaN
3,100028,NaN,0,0,1,2,315000.0,1575000.0,49018.5,1575000.0,...,0,1,0,1,1,0,0,1,1,NaN
4,100038,NaN,1,1,0,1,180000.0,625500.0,32067.0,625500.0,...,0,1,0,1,0,0,0,0,0,NaN


In [22]:
train_setB1xgb = pd.read_csv('./home-credit-default-risk/exports/train_setB1xgb.csv')
test_setB1xgb =  pd.read_csv('./home-credit-default-risk/exports/test_setB1xgb.csv')
test_setB1xgb.head()

,SK_ID_CURR,setB1x
0,100001,0.032762
1,100005,0.100620
2,100013,0.044999
3,100028,0.034211
4,100038,0.172532


In [23]:
app_train = app_train.merge(train_setB1xgb, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_setB1xgb, how='left', on='SK_ID_CURR')
app_test.head()

,SK_ID_CURR,TARGET,APP_CODE_GENDER,APP_FLAG_OWN_CAR,APP_FLAG_OWN_REALTY,APP_CNT_CHILDREN,APP_AMT_INCOME_TOTAL,APP_AMT_CREDIT,APP_AMT_ANNUITY,APP_AMT_GOODS_PRICE,...,PREV_NAME_SELLER_INDUSTRY_Construction_SUM_high_risk,PREV_NAME_SELLER_INDUSTRY_Furniture_SUM_low_risk,PREV_NAME_SELLER_INDUSTRY_Industry_SUM_high_risk,CCB_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CCB_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,BB_CNT_CREDIT_PROLONG_MAX_high_risk,BB_CNT_CREDIT_PROLONG_MAX_low_risk,BB_NEW_ACTIVE_CNT_CREDIT_PROLONG_SUM_low_risk,setB1a,setB1x
0,100001,NaN,0,0,1,0,135000.0,568800.0,20560.5,450000.0,...,1,0,1,0,0,0,1,1,NaN,0.032762
1,100005,NaN,1,0,1,0,99000.0,222768.0,17370.0,180000.0,...,1,0,1,0,0,0,1,1,NaN,0.100620
2,100013,NaN,1,1,1,0,202500.0,663264.0,69777.0,630000.0,...,1,0,1,1,0,0,1,0,NaN,0.044999
3,100028,NaN,0,0,1,2,315000.0,1575000.0,49018.5,1575000.0,...,1,0,1,1,0,0,1,1,NaN,0.034211
4,100038,NaN,1,1,0,1,180000.0,625500.0,32067.0,625500.0,...,1,0,1,0,0,0,0,0,NaN,0.172532


In [24]:
pred_train_set1a = pd.read_csv('./home-credit-default-risk/exports/pred_train_set1a.csv')
pred_test_set1a =  pd.read_csv('./home-credit-default-risk/exports/pred_test_set1a.csv')
pred_test_set1a.head()

,SK_ID_CURR,set1
0,100001,0.040068
1,100005,0.095240
2,100013,0.044435
3,100028,0.036586
4,100038,0.141754


In [25]:
app_train = app_train.merge(pred_train_set1a, how='left', on='SK_ID_CURR')
app_test = app_test.merge(pred_test_set1a, how='left', on='SK_ID_CURR')
app_test.head()

,SK_ID_CURR,TARGET,APP_CODE_GENDER,APP_FLAG_OWN_CAR,APP_FLAG_OWN_REALTY,APP_CNT_CHILDREN,APP_AMT_INCOME_TOTAL,APP_AMT_CREDIT,APP_AMT_ANNUITY,APP_AMT_GOODS_PRICE,...,PREV_NAME_SELLER_INDUSTRY_Furniture_SUM_low_risk,PREV_NAME_SELLER_INDUSTRY_Industry_SUM_high_risk,CCB_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CCB_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,BB_CNT_CREDIT_PROLONG_MAX_high_risk,BB_CNT_CREDIT_PROLONG_MAX_low_risk,BB_NEW_ACTIVE_CNT_CREDIT_PROLONG_SUM_low_risk,setB1a,setB1x,set1
0,100001,NaN,0,0,1,0,135000.0,568800.0,20560.5,450000.0,...,0,1,0,0,0,1,1,NaN,0.032762,0.040068
1,100005,NaN,1,0,1,0,99000.0,222768.0,17370.0,180000.0,...,0,1,0,0,0,1,1,NaN,0.100620,0.095240
2,100013,NaN,1,1,1,0,202500.0,663264.0,69777.0,630000.0,...,0,1,1,0,0,1,0,NaN,0.044999,0.044435
3,100028,NaN,0,0,1,2,315000.0,1575000.0,49018.5,1575000.0,...,0,1,1,0,0,1,1,NaN,0.034211,0.036586
4,100038,NaN,1,1,0,1,180000.0,625500.0,32067.0,625500.0,...,0,1,0,0,0,0,0,NaN,0.172532,0.141754


In [26]:
train_set1xgb = pd.read_csv('./home-credit-default-risk/exports/train_set1xgb.csv')
test_set1xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set1xgb.csv')
test_set1xgb.head()

,SK_ID_CURR,set1x
0,100001,0.035234
1,100005,0.107426
2,100013,0.048029
3,100028,0.032975
4,100038,0.161962


In [27]:
app_train = app_train.merge(train_set1xgb, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set1xgb, how='left', on='SK_ID_CURR')
app_test.head()

,SK_ID_CURR,TARGET,APP_CODE_GENDER,APP_FLAG_OWN_CAR,APP_FLAG_OWN_REALTY,APP_CNT_CHILDREN,APP_AMT_INCOME_TOTAL,APP_AMT_CREDIT,APP_AMT_ANNUITY,APP_AMT_GOODS_PRICE,...,PREV_NAME_SELLER_INDUSTRY_Industry_SUM_high_risk,CCB_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CCB_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,BB_CNT_CREDIT_PROLONG_MAX_high_risk,BB_CNT_CREDIT_PROLONG_MAX_low_risk,BB_NEW_ACTIVE_CNT_CREDIT_PROLONG_SUM_low_risk,setB1a,setB1x,set1,set1x
0,100001,NaN,0,0,1,0,135000.0,568800.0,20560.5,450000.0,...,1,0,0,0,1,1,NaN,0.032762,0.040068,0.035234
1,100005,NaN,1,0,1,0,99000.0,222768.0,17370.0,180000.0,...,1,0,0,0,1,1,NaN,0.100620,0.095240,0.107426
2,100013,NaN,1,1,1,0,202500.0,663264.0,69777.0,630000.0,...,1,1,0,0,1,0,NaN,0.044999,0.044435,0.048029
3,100028,NaN,0,0,1,2,315000.0,1575000.0,49018.5,1575000.0,...,1,1,0,0,1,1,NaN,0.034211,0.036586,0.032975
4,100038,NaN,1,1,0,1,180000.0,625500.0,32067.0,625500.0,...,1,0,0,0,0,0,NaN,0.172532,0.141754,0.161962


In [28]:
train_set2a = pd.read_csv('./home-credit-default-risk/exports/train_set2a.csv')
test_set2a =  pd.read_csv('./home-credit-default-risk/exports/test_set2a.csv')
test_set2a.head()

,SK_ID_CURR,set2a
0,100001,0.042769
1,100005,0.103694
2,100013,0.036006
3,100028,0.048812
4,100038,0.173045


In [29]:
app_train = app_train.merge(train_set2a, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set2a, how='left', on='SK_ID_CURR')
app_test.head()

,SK_ID_CURR,TARGET,APP_CODE_GENDER,APP_FLAG_OWN_CAR,APP_FLAG_OWN_REALTY,APP_CNT_CHILDREN,APP_AMT_INCOME_TOTAL,APP_AMT_CREDIT,APP_AMT_ANNUITY,APP_AMT_GOODS_PRICE,...,CCB_CNT_DRAWINGS_OTHER_CURRENT_MAX_high_risk,CCB_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,BB_CNT_CREDIT_PROLONG_MAX_high_risk,BB_CNT_CREDIT_PROLONG_MAX_low_risk,BB_NEW_ACTIVE_CNT_CREDIT_PROLONG_SUM_low_risk,setB1a,setB1x,set1,set1x,set2a
0,100001,NaN,0,0,1,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,1,1,NaN,0.032762,0.040068,0.035234,0.042769
1,100005,NaN,1,0,1,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,1,1,NaN,0.100620,0.095240,0.107426,0.103694
2,100013,NaN,1,1,1,0,202500.0,663264.0,69777.0,630000.0,...,1,0,0,1,0,NaN,0.044999,0.044435,0.048029,0.036006
3,100028,NaN,0,0,1,2,315000.0,1575000.0,49018.5,1575000.0,...,1,0,0,1,1,NaN,0.034211,0.036586,0.032975,0.048812
4,100038,NaN,1,1,0,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,0,NaN,0.172532,0.141754,0.161962,0.173045


In [30]:
train_set2xgb = pd.read_csv('./home-credit-default-risk/exports/train_set2xgb.csv')
test_set2xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set2xgb.csv')
test_set2xgb.head()

,SK_ID_CURR,set2x
0,100001,0.035013
1,100005,0.103532
2,100013,0.048415
3,100028,0.032249
4,100038,0.150442


In [31]:
app_train = app_train.merge(train_set2xgb, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set2xgb, how='left', on='SK_ID_CURR')
app_test.head()

,SK_ID_CURR,TARGET,APP_CODE_GENDER,APP_FLAG_OWN_CAR,APP_FLAG_OWN_REALTY,APP_CNT_CHILDREN,APP_AMT_INCOME_TOTAL,APP_AMT_CREDIT,APP_AMT_ANNUITY,APP_AMT_GOODS_PRICE,...,CCB_CNT_DRAWINGS_OTHER_CURRENT_MAX_low_risk,BB_CNT_CREDIT_PROLONG_MAX_high_risk,BB_CNT_CREDIT_PROLONG_MAX_low_risk,BB_NEW_ACTIVE_CNT_CREDIT_PROLONG_SUM_low_risk,setB1a,setB1x,set1,set1x,set2a,set2x
0,100001,NaN,0,0,1,0,135000.0,568800.0,20560.5,450000.0,...,0,0,1,1,NaN,0.032762,0.040068,0.035234,0.042769,0.035013
1,100005,NaN,1,0,1,0,99000.0,222768.0,17370.0,180000.0,...,0,0,1,1,NaN,0.100620,0.095240,0.107426,0.103694,0.103532
2,100013,NaN,1,1,1,0,202500.0,663264.0,69777.0,630000.0,...,0,0,1,0,NaN,0.044999,0.044435,0.048029,0.036006,0.048415
3,100028,NaN,0,0,1,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,1,1,NaN,0.034211,0.036586,0.032975,0.048812,0.032249
4,100038,NaN,1,1,0,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,0.172532,0.141754,0.161962,0.173045,0.150442


In [32]:
train_set2nn = pd.read_csv('./home-credit-default-risk/exports/train_set2nn.csv')
test_set2nn =  pd.read_csv('./home-credit-default-risk/exports/test_set2nn.csv')
test_set2nn.head()

,SK_ID_CURR,set2nn
0,100001,0.048972
1,100005,0.141351
2,100013,0.028088
3,100028,0.039056
4,100038,0.192637


In [33]:
app_train = app_train.merge(train_set2nn, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set2nn, how='left', on='SK_ID_CURR')
app_test.head()

,SK_ID_CURR,TARGET,APP_CODE_GENDER,APP_FLAG_OWN_CAR,APP_FLAG_OWN_REALTY,APP_CNT_CHILDREN,APP_AMT_INCOME_TOTAL,APP_AMT_CREDIT,APP_AMT_ANNUITY,APP_AMT_GOODS_PRICE,...,BB_CNT_CREDIT_PROLONG_MAX_high_risk,BB_CNT_CREDIT_PROLONG_MAX_low_risk,BB_NEW_ACTIVE_CNT_CREDIT_PROLONG_SUM_low_risk,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn
0,100001,NaN,0,0,1,0,135000.0,568800.0,20560.5,450000.0,...,0,1,1,NaN,0.032762,0.040068,0.035234,0.042769,0.035013,0.048972
1,100005,NaN,1,0,1,0,99000.0,222768.0,17370.0,180000.0,...,0,1,1,NaN,0.100620,0.095240,0.107426,0.103694,0.103532,0.141351
2,100013,NaN,1,1,1,0,202500.0,663264.0,69777.0,630000.0,...,0,1,0,NaN,0.044999,0.044435,0.048029,0.036006,0.048415,0.028088
3,100028,NaN,0,0,1,2,315000.0,1575000.0,49018.5,1575000.0,...,0,1,1,NaN,0.034211,0.036586,0.032975,0.048812,0.032249,0.039056
4,100038,NaN,1,1,0,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,NaN,0.172532,0.141754,0.161962,0.173045,0.150442,0.192637


In [34]:
train_set3a = pd.read_csv('./home-credit-default-risk/exports/train_set3a.csv')
test_set3a =  pd.read_csv('./home-credit-default-risk/exports/test_set3a.csv')
test_set3a.head()

,SK_ID_CURR,set3a
0,100001,0.049120
1,100005,0.115729
2,100013,0.064166
3,100028,0.038931
4,100038,0.195575


In [35]:
app_train = app_train.merge(train_set3a, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set3a, how='left', on='SK_ID_CURR')
app_test.head()

,SK_ID_CURR,TARGET,APP_CODE_GENDER,APP_FLAG_OWN_CAR,APP_FLAG_OWN_REALTY,APP_CNT_CHILDREN,APP_AMT_INCOME_TOTAL,APP_AMT_CREDIT,APP_AMT_ANNUITY,APP_AMT_GOODS_PRICE,...,BB_CNT_CREDIT_PROLONG_MAX_low_risk,BB_NEW_ACTIVE_CNT_CREDIT_PROLONG_SUM_low_risk,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn,set3a
0,100001,NaN,0,0,1,0,135000.0,568800.0,20560.5,450000.0,...,1,1,NaN,0.032762,0.040068,0.035234,0.042769,0.035013,0.048972,0.049120
1,100005,NaN,1,0,1,0,99000.0,222768.0,17370.0,180000.0,...,1,1,NaN,0.100620,0.095240,0.107426,0.103694,0.103532,0.141351,0.115729
2,100013,NaN,1,1,1,0,202500.0,663264.0,69777.0,630000.0,...,1,0,NaN,0.044999,0.044435,0.048029,0.036006,0.048415,0.028088,0.064166
3,100028,NaN,0,0,1,2,315000.0,1575000.0,49018.5,1575000.0,...,1,1,NaN,0.034211,0.036586,0.032975,0.048812,0.032249,0.039056,0.038931
4,100038,NaN,1,1,0,1,180000.0,625500.0,32067.0,625500.0,...,0,0,NaN,0.172532,0.141754,0.161962,0.173045,0.150442,0.192637,0.195575


In [36]:
train_set3xgb = pd.read_csv('./home-credit-default-risk/exports/train_set3xgb.csv')
test_set3xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set3xgb.csv')
test_set3xgb.head()

,SK_ID_CURR,set3x
0,100001,0.034310
1,100005,0.127556
2,100013,0.044872
3,100028,0.030915
4,100038,0.201295


In [37]:
app_train = app_train.merge(train_set3xgb, how='left', on='SK_ID_CURR')
app_test = app_test.merge(test_set3xgb, how='left', on='SK_ID_CURR')
app_test.head()

,SK_ID_CURR,TARGET,APP_CODE_GENDER,APP_FLAG_OWN_CAR,APP_FLAG_OWN_REALTY,APP_CNT_CHILDREN,APP_AMT_INCOME_TOTAL,APP_AMT_CREDIT,APP_AMT_ANNUITY,APP_AMT_GOODS_PRICE,...,BB_NEW_ACTIVE_CNT_CREDIT_PROLONG_SUM_low_risk,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn,set3a,set3x
0,100001,NaN,0,0,1,0,135000.0,568800.0,20560.5,450000.0,...,1,NaN,0.032762,0.040068,0.035234,0.042769,0.035013,0.048972,0.049120,0.034310
1,100005,NaN,1,0,1,0,99000.0,222768.0,17370.0,180000.0,...,1,NaN,0.100620,0.095240,0.107426,0.103694,0.103532,0.141351,0.115729,0.127556
2,100013,NaN,1,1,1,0,202500.0,663264.0,69777.0,630000.0,...,0,NaN,0.044999,0.044435,0.048029,0.036006,0.048415,0.028088,0.064166,0.044872
3,100028,NaN,0,0,1,2,315000.0,1575000.0,49018.5,1575000.0,...,1,NaN,0.034211,0.036586,0.032975,0.048812,0.032249,0.039056,0.038931,0.030915
4,100038,NaN,1,1,0,1,180000.0,625500.0,32067.0,625500.0,...,0,NaN,0.172532,0.141754,0.161962,0.173045,0.150442,0.192637,0.195575,0.201295


In [38]:
### indexを削除
app_train=app_train.reset_index()
app_test=app_test.reset_index()
app_test.head()

,index,SK_ID_CURR,TARGET,APP_CODE_GENDER,APP_FLAG_OWN_CAR,APP_FLAG_OWN_REALTY,APP_CNT_CHILDREN,APP_AMT_INCOME_TOTAL,APP_AMT_CREDIT,APP_AMT_ANNUITY,...,BB_NEW_ACTIVE_CNT_CREDIT_PROLONG_SUM_low_risk,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn,set3a,set3x
0,0,100001,NaN,0,0,1,0,135000.0,568800.0,20560.5,...,1,NaN,0.032762,0.040068,0.035234,0.042769,0.035013,0.048972,0.049120,0.034310
1,1,100005,NaN,1,0,1,0,99000.0,222768.0,17370.0,...,1,NaN,0.100620,0.095240,0.107426,0.103694,0.103532,0.141351,0.115729,0.127556
2,2,100013,NaN,1,1,1,0,202500.0,663264.0,69777.0,...,0,NaN,0.044999,0.044435,0.048029,0.036006,0.048415,0.028088,0.064166,0.044872
3,3,100028,NaN,0,0,1,2,315000.0,1575000.0,49018.5,...,1,NaN,0.034211,0.036586,0.032975,0.048812,0.032249,0.039056,0.038931,0.030915
4,4,100038,NaN,1,1,0,1,180000.0,625500.0,32067.0,...,0,NaN,0.172532,0.141754,0.161962,0.173045,0.150442,0.192637,0.195575,0.201295


In [39]:
### indexを削除
app_train = app_train.drop(columns=['index'])
app_test = app_test.drop(columns=['index'])
app_test.head()

,SK_ID_CURR,TARGET,APP_CODE_GENDER,APP_FLAG_OWN_CAR,APP_FLAG_OWN_REALTY,APP_CNT_CHILDREN,APP_AMT_INCOME_TOTAL,APP_AMT_CREDIT,APP_AMT_ANNUITY,APP_AMT_GOODS_PRICE,...,BB_NEW_ACTIVE_CNT_CREDIT_PROLONG_SUM_low_risk,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn,set3a,set3x
0,100001,NaN,0,0,1,0,135000.0,568800.0,20560.5,450000.0,...,1,NaN,0.032762,0.040068,0.035234,0.042769,0.035013,0.048972,0.049120,0.034310
1,100005,NaN,1,0,1,0,99000.0,222768.0,17370.0,180000.0,...,1,NaN,0.100620,0.095240,0.107426,0.103694,0.103532,0.141351,0.115729,0.127556
2,100013,NaN,1,1,1,0,202500.0,663264.0,69777.0,630000.0,...,0,NaN,0.044999,0.044435,0.048029,0.036006,0.048415,0.028088,0.064166,0.044872
3,100028,NaN,0,0,1,2,315000.0,1575000.0,49018.5,1575000.0,...,1,NaN,0.034211,0.036586,0.032975,0.048812,0.032249,0.039056,0.038931,0.030915
4,100038,NaN,1,1,0,1,180000.0,625500.0,32067.0,625500.0,...,0,NaN,0.172532,0.141754,0.161962,0.173045,0.150442,0.192637,0.195575,0.201295


In [40]:
# infをnanへ置換
app_train = app_train.replace([np.inf, -np.inf], np.nan)
app_test = app_test.replace([np.inf, -np.inf], np.nan)
app_test.head()

,SK_ID_CURR,TARGET,APP_CODE_GENDER,APP_FLAG_OWN_CAR,APP_FLAG_OWN_REALTY,APP_CNT_CHILDREN,APP_AMT_INCOME_TOTAL,APP_AMT_CREDIT,APP_AMT_ANNUITY,APP_AMT_GOODS_PRICE,...,BB_NEW_ACTIVE_CNT_CREDIT_PROLONG_SUM_low_risk,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn,set3a,set3x
0,100001,NaN,0,0,1,0,135000.0,568800.0,20560.5,450000.0,...,1,NaN,0.032762,0.040068,0.035234,0.042769,0.035013,0.048972,0.049120,0.034310
1,100005,NaN,1,0,1,0,99000.0,222768.0,17370.0,180000.0,...,1,NaN,0.100620,0.095240,0.107426,0.103694,0.103532,0.141351,0.115729,0.127556
2,100013,NaN,1,1,1,0,202500.0,663264.0,69777.0,630000.0,...,0,NaN,0.044999,0.044435,0.048029,0.036006,0.048415,0.028088,0.064166,0.044872
3,100028,NaN,0,0,1,2,315000.0,1575000.0,49018.5,1575000.0,...,1,NaN,0.034211,0.036586,0.032975,0.048812,0.032249,0.039056,0.038931,0.030915
4,100038,NaN,1,1,0,1,180000.0,625500.0,32067.0,625500.0,...,0,NaN,0.172532,0.141754,0.161962,0.173045,0.150442,0.192637,0.195575,0.201295


In [41]:
### 処理対象の列名を配列化
encoding_columns = list(app_test.select_dtypes(include='number').columns)
encoding_columns.remove('SK_ID_CURR')
encoding_columns.remove('TARGET')

In [42]:
### indexを設定
app_train = app_train.set_index('SK_ID_CURR')
app_test = app_test.set_index('SK_ID_CURR')
app_test.head()

,TARGET,APP_CODE_GENDER,APP_FLAG_OWN_CAR,APP_FLAG_OWN_REALTY,APP_CNT_CHILDREN,APP_AMT_INCOME_TOTAL,APP_AMT_CREDIT,APP_AMT_ANNUITY,APP_AMT_GOODS_PRICE,APP_REGION_POPULATION_RELATIVE,...,BB_NEW_ACTIVE_CNT_CREDIT_PROLONG_SUM_low_risk,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn,set3a,set3x
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,NaN,0,0,1,0,135000.0,568800.0,20560.5,450000.0,0.018845,...,1,NaN,0.032762,0.040068,0.035234,0.042769,0.035013,0.048972,0.049120,0.034310
100005,NaN,1,0,1,0,99000.0,222768.0,17370.0,180000.0,0.035797,...,1,NaN,0.100620,0.095240,0.107426,0.103694,0.103532,0.141351,0.115729,0.127556
100013,NaN,1,1,1,0,202500.0,663264.0,69777.0,630000.0,0.019104,...,0,NaN,0.044999,0.044435,0.048029,0.036006,0.048415,0.028088,0.064166,0.044872
100028,NaN,0,0,1,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,...,1,NaN,0.034211,0.036586,0.032975,0.048812,0.032249,0.039056,0.038931,0.030915
100038,NaN,1,1,0,1,180000.0,625500.0,32067.0,625500.0,0.010033,...,0,NaN,0.172532,0.141754,0.161962,0.173045,0.150442,0.192637,0.195575,0.201295


In [43]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(app_train, app_test, num_cols)
        
app_test.head()

,TARGET,APP_CODE_GENDER,APP_FLAG_OWN_CAR,APP_FLAG_OWN_REALTY,APP_CNT_CHILDREN,APP_AMT_INCOME_TOTAL,APP_AMT_CREDIT,APP_AMT_ANNUITY,APP_AMT_GOODS_PRICE,APP_REGION_POPULATION_RELATIVE,...,BB_NEW_ACTIVE_CNT_CREDIT_PROLONG_SUM_low_risk,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn,set3a,set3x
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,NaN,-0.720377,-0.717910,0.664538,-0.577534,-0.142532,-0.075104,-0.451792,-0.239271,-0.146484,...,0.667432,NaN,-0.485896,-0.419361,-0.450891,-0.402589,-0.457814,-0.317197,-0.328307,-0.469542
100005,NaN,1.388162,-0.717910,0.664538,-0.577534,-0.294351,-0.934828,-0.671921,-0.970093,1.079102,...,0.667432,NaN,0.226294,0.159145,0.319825,0.256510,0.261749,0.678170,0.365058,0.507163
100013,NaN,1.388162,1.392932,0.664538,-0.577534,0.142129,0.159594,2.943907,0.247945,-0.127563,...,-1.498279,NaN,-0.357465,-0.373566,-0.314289,-0.475758,-0.317065,-0.542220,-0.171688,-0.358904
100028,NaN,-0.720377,-0.717910,0.664538,2.192098,0.616564,2.424822,1.511672,2.805824,0.399658,...,0.667432,NaN,-0.470693,-0.455871,-0.475004,-0.337219,-0.486834,-0.424040,-0.434369,-0.505095
100038,NaN,1.388162,1.392932,-1.504806,0.807282,0.047242,0.065769,0.342101,0.235764,-0.783203,...,-1.498279,NaN,0.981029,0.646858,0.902041,1.006761,0.754380,1.230772,1.196224,1.279548


In [44]:
### indexを削除
app_train = app_train.reset_index()
app_test = app_test.reset_index()
app_test.head()

,SK_ID_CURR,TARGET,APP_CODE_GENDER,APP_FLAG_OWN_CAR,APP_FLAG_OWN_REALTY,APP_CNT_CHILDREN,APP_AMT_INCOME_TOTAL,APP_AMT_CREDIT,APP_AMT_ANNUITY,APP_AMT_GOODS_PRICE,...,BB_NEW_ACTIVE_CNT_CREDIT_PROLONG_SUM_low_risk,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn,set3a,set3x
0,100001,NaN,-0.720377,-0.717910,0.664538,-0.577534,-0.142532,-0.075104,-0.451792,-0.239271,...,0.667432,NaN,-0.485896,-0.419361,-0.450891,-0.402589,-0.457814,-0.317197,-0.328307,-0.469542
1,100005,NaN,1.388162,-0.717910,0.664538,-0.577534,-0.294351,-0.934828,-0.671921,-0.970093,...,0.667432,NaN,0.226294,0.159145,0.319825,0.256510,0.261749,0.678170,0.365058,0.507163
2,100013,NaN,1.388162,1.392932,0.664538,-0.577534,0.142129,0.159594,2.943907,0.247945,...,-1.498279,NaN,-0.357465,-0.373566,-0.314289,-0.475758,-0.317065,-0.542220,-0.171688,-0.358904
3,100028,NaN,-0.720377,-0.717910,0.664538,2.192098,0.616564,2.424822,1.511672,2.805824,...,0.667432,NaN,-0.470693,-0.455871,-0.475004,-0.337219,-0.486834,-0.424040,-0.434369,-0.505095
4,100038,NaN,1.388162,1.392932,-1.504806,0.807282,0.047242,0.065769,0.342101,0.235764,...,-1.498279,NaN,0.981029,0.646858,0.902041,1.006761,0.754380,1.230772,1.196224,1.279548


In [45]:
app_train = FeatureEngineering2.reduce_mem_usage(app_train)
app_train.head()

,SK_ID_CURR,TARGET,APP_CODE_GENDER,APP_FLAG_OWN_CAR,APP_FLAG_OWN_REALTY,APP_CNT_CHILDREN,APP_AMT_INCOME_TOTAL,APP_AMT_CREDIT,APP_AMT_ANNUITY,APP_AMT_GOODS_PRICE,...,BB_NEW_ACTIVE_CNT_CREDIT_PROLONG_SUM_low_risk,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn,set3a,set3x
0,100002,1.0,1.387695,-0.717773,0.664551,-0.577637,0.142090,-0.478027,-0.166138,-0.507324,...,0.667480,2.107422,1.634766,1.896484,1.301758,1.757812,1.261719,1.746094,1.488281,1.888672
1,100003,0.0,-0.720215,-0.717773,-1.504883,-0.577637,0.426758,1.725586,0.592773,1.599609,...,0.667480,-0.598145,-0.587891,-0.711914,-0.698242,-0.671875,-0.692871,-0.686035,-0.588379,-0.587402
2,100004,0.0,1.387695,1.392578,0.664551,-0.577637,-0.427246,-1.153320,-1.404297,-1.091797,...,-1.498047,-0.501465,-0.423340,-0.420654,-0.320557,-0.505859,-0.478271,-0.523926,-0.471924,-0.480225
3,100006,0.0,-0.720215,-0.717773,0.664551,-0.577637,-0.142578,-0.711426,0.177856,-0.653320,...,-1.498047,-0.541016,-0.472900,-0.516602,-0.505859,-0.336182,-0.558105,-0.501953,-0.515625,-0.589355
4,100007,0.0,1.387695,-0.717773,0.664551,-0.577637,-0.199463,-0.213745,-0.361816,-0.068726,...,-1.498047,-0.596191,-0.537598,-0.471680,-0.575684,-0.394043,-0.414062,-0.188232,-0.473633,-0.481934


In [46]:
app_test = FeatureEngineering2.reduce_mem_usage(app_test)
app_test.head()

,SK_ID_CURR,TARGET,APP_CODE_GENDER,APP_FLAG_OWN_CAR,APP_FLAG_OWN_REALTY,APP_CNT_CHILDREN,APP_AMT_INCOME_TOTAL,APP_AMT_CREDIT,APP_AMT_ANNUITY,APP_AMT_GOODS_PRICE,...,BB_NEW_ACTIVE_CNT_CREDIT_PROLONG_SUM_low_risk,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn,set3a,set3x
0,100001,NaN,-0.720215,-0.717773,0.664551,-0.577637,-0.142578,-0.075073,-0.451904,-0.239258,...,0.667480,NaN,-0.485840,-0.419434,-0.450928,-0.402588,-0.457764,-0.317139,-0.328369,-0.469482
1,100005,NaN,1.387695,-0.717773,0.664551,-0.577637,-0.294434,-0.935059,-0.671875,-0.970215,...,0.667480,NaN,0.226318,0.159180,0.319824,0.256592,0.261719,0.678223,0.364990,0.507324
2,100013,NaN,1.387695,1.392578,0.664551,-0.577637,0.142090,0.159546,2.943359,0.247925,...,-1.498047,NaN,-0.357422,-0.373535,-0.314209,-0.475830,-0.317139,-0.541992,-0.171631,-0.358887
3,100028,NaN,-0.720215,-0.717773,0.664551,2.191406,0.616699,2.425781,1.511719,2.806641,...,0.667480,NaN,-0.470703,-0.455811,-0.475098,-0.337158,-0.486816,-0.424072,-0.434326,-0.504883
4,100038,NaN,1.387695,1.392578,-1.504883,0.807129,0.047241,0.065796,0.342041,0.235718,...,-1.498047,NaN,0.980957,0.646973,0.901855,1.006836,0.754395,1.230469,1.196289,1.279297


In [47]:
#app_train.to_csv(
#    path_or_buf="./app_test_set4.csv" # 出力先
#    ,index=None                                        # indexの出力有無
#)

In [48]:
# trainデータセット作成
x_train = app_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train['TARGET']
id_train = app_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [49]:
# testデータセット作成
x_test = app_test.drop(columns=['SK_ID_CURR', 'TARGET'])
id_test = app_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [50]:
### training Xgb
# 1層目のモデル
# pred_train_1a, pred_train_1bは、学習データのクロスバリデーションでの予測値
# pred_test_1a, pred_test_1bは、テストデータの予測値
model_1a = Model1Xgb()
pred_train_1a, pred_test_1a = predict_cv(model_1a, x_train, y_train, x_test)

[09:18:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "num_round", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-auc:0.78416	eval-auc:0.78493
[1]	train-auc:0.79151	eval-auc:0.79092
[2]	train-auc:0.79419	eval-auc:0.79332
[3]	train-auc:0.79481	eval-auc:0.79368
[4]	train-auc:0.79506	eval-auc:0.79387
[5]	train-auc:0.79547	eval-auc:0.79420
[6]	train-auc:0.79585	eval-auc:0.79446
[7]	train-auc:0.79609	eval-auc:0.79469
[8]	train-auc:0.79659	eval-auc:0.79488
[9]	train-auc:0.79650	eval-auc:0.79485
[10]	train-auc:0.79685	eval-auc:0.79554
[11]	train-auc:0.79697	eval-auc:0.79540
[12]	train-auc:0.79709	eval-auc:0.79538
[13]	train-auc:0.79726	eval-auc:0.79552
[14

[192]	train-auc:0.81047	eval-auc:0.79662
[193]	train-auc:0.81056	eval-auc:0.79662
[194]	train-auc:0.81068	eval-auc:0.79664
[195]	train-auc:0.81081	eval-auc:0.79665
[196]	train-auc:0.81091	eval-auc:0.79666
[197]	train-auc:0.81107	eval-auc:0.79669
[198]	train-auc:0.81116	eval-auc:0.79668
[199]	train-auc:0.81124	eval-auc:0.79667
[200]	train-auc:0.81139	eval-auc:0.79667
[201]	train-auc:0.81147	eval-auc:0.79667
[202]	train-auc:0.81160	eval-auc:0.79666
[203]	train-auc:0.81170	eval-auc:0.79667
[204]	train-auc:0.81182	eval-auc:0.79665
[205]	train-auc:0.81193	eval-auc:0.79665
[206]	train-auc:0.81205	eval-auc:0.79666
[207]	train-auc:0.81221	eval-auc:0.79664
[208]	train-auc:0.81234	eval-auc:0.79663
[209]	train-auc:0.81250	eval-auc:0.79663
[210]	train-auc:0.81264	eval-auc:0.79663
[211]	train-auc:0.81277	eval-auc:0.79663
[212]	train-auc:0.81294	eval-auc:0.79663
[213]	train-auc:0.81307	eval-auc:0.79664
[214]	train-auc:0.81317	eval-auc:0.79663
[215]	train-auc:0.81330	eval-auc:0.79661
[216]	train-auc:

[392]	train-auc:0.84250	eval-auc:0.79654
[393]	train-auc:0.84275	eval-auc:0.79656
[394]	train-auc:0.84288	eval-auc:0.79655
[395]	train-auc:0.84307	eval-auc:0.79660
[396]	train-auc:0.84325	eval-auc:0.79659
[397]	train-auc:0.84340	eval-auc:0.79657
[398]	train-auc:0.84365	eval-auc:0.79658
[399]	train-auc:0.84379	eval-auc:0.79658
[400]	train-auc:0.84396	eval-auc:0.79658
[401]	train-auc:0.84412	eval-auc:0.79656
[402]	train-auc:0.84430	eval-auc:0.79656
[403]	train-auc:0.84451	eval-auc:0.79657
[404]	train-auc:0.84464	eval-auc:0.79657
[405]	train-auc:0.84488	eval-auc:0.79656
[406]	train-auc:0.84505	eval-auc:0.79655
[407]	train-auc:0.84525	eval-auc:0.79655
[408]	train-auc:0.84544	eval-auc:0.79656
[409]	train-auc:0.84562	eval-auc:0.79659
[410]	train-auc:0.84585	eval-auc:0.79660
[411]	train-auc:0.84601	eval-auc:0.79661
[412]	train-auc:0.84623	eval-auc:0.79662
[413]	train-auc:0.84641	eval-auc:0.79662
[414]	train-auc:0.84662	eval-auc:0.79665
[415]	train-auc:0.84680	eval-auc:0.79664
[416]	train-auc:

[592]	train-auc:0.87465	eval-auc:0.79645
[593]	train-auc:0.87474	eval-auc:0.79644
[594]	train-auc:0.87487	eval-auc:0.79644
[595]	train-auc:0.87506	eval-auc:0.79642
[596]	train-auc:0.87523	eval-auc:0.79643
[597]	train-auc:0.87536	eval-auc:0.79644
[598]	train-auc:0.87552	eval-auc:0.79642
[599]	train-auc:0.87568	eval-auc:0.79642
[600]	train-auc:0.87580	eval-auc:0.79643
[601]	train-auc:0.87597	eval-auc:0.79643
[602]	train-auc:0.87599	eval-auc:0.79644
[603]	train-auc:0.87608	eval-auc:0.79643
[604]	train-auc:0.87625	eval-auc:0.79643
[605]	train-auc:0.87639	eval-auc:0.79642
[606]	train-auc:0.87652	eval-auc:0.79641
[607]	train-auc:0.87659	eval-auc:0.79642
[608]	train-auc:0.87669	eval-auc:0.79641
[609]	train-auc:0.87678	eval-auc:0.79642
[610]	train-auc:0.87692	eval-auc:0.79643
[611]	train-auc:0.87712	eval-auc:0.79643
[612]	train-auc:0.87729	eval-auc:0.79643
[613]	train-auc:0.87743	eval-auc:0.79643
[614]	train-auc:0.87755	eval-auc:0.79643
[615]	train-auc:0.87762	eval-auc:0.79643
[616]	train-auc:

[792]	train-auc:0.89865	eval-auc:0.79618
[793]	train-auc:0.89879	eval-auc:0.79617
[794]	train-auc:0.89893	eval-auc:0.79619
[795]	train-auc:0.89905	eval-auc:0.79618
[796]	train-auc:0.89916	eval-auc:0.79617
[797]	train-auc:0.89934	eval-auc:0.79617
[798]	train-auc:0.89944	eval-auc:0.79616
[799]	train-auc:0.89965	eval-auc:0.79614
[800]	train-auc:0.89977	eval-auc:0.79615
[801]	train-auc:0.89989	eval-auc:0.79615
[802]	train-auc:0.90004	eval-auc:0.79614
[803]	train-auc:0.90012	eval-auc:0.79613
[804]	train-auc:0.90027	eval-auc:0.79613
[805]	train-auc:0.90038	eval-auc:0.79612
[806]	train-auc:0.90051	eval-auc:0.79611
[807]	train-auc:0.90061	eval-auc:0.79611
[808]	train-auc:0.90075	eval-auc:0.79611
[809]	train-auc:0.90086	eval-auc:0.79608
[810]	train-auc:0.90099	eval-auc:0.79607
[811]	train-auc:0.90115	eval-auc:0.79608
[812]	train-auc:0.90123	eval-auc:0.79608
[813]	train-auc:0.90135	eval-auc:0.79607
[814]	train-auc:0.90148	eval-auc:0.79604
[815]	train-auc:0.90161	eval-auc:0.79603
[816]	train-auc:

...

[160]	train-auc:0.80699	eval-auc:0.79751
[161]	train-auc:0.80709	eval-auc:0.79752
[162]	train-auc:0.80715	eval-auc:0.79750
[163]	train-auc:0.80726	eval-auc:0.79751
[164]	train-auc:0.80733	eval-auc:0.79757
[165]	train-auc:0.80746	eval-auc:0.79758
[166]	train-auc:0.80757	eval-auc:0.79757
[167]	train-auc:0.80769	eval-auc:0.79758
[168]	train-auc:0.80777	eval-auc:0.79758
[169]	train-auc:0.80792	eval-auc:0.79757
[170]	train-auc:0.80802	eval-auc:0.79757
[171]	train-auc:0.80813	eval-auc:0.79757
[172]	train-auc:0.80820	eval-auc:0.79758
[173]	train-auc:0.80829	eval-auc:0.79760
[174]	train-auc:0.80840	eval-auc:0.79760
[175]	train-auc:0.80853	eval-auc:0.79760
[176]	train-auc:0.80861	eval-auc:0.79760
[177]	train-auc:0.80872	eval-auc:0.79761
[178]	train-auc:0.80886	eval-auc:0.79761
[179]	train-auc:0.80898	eval-auc:0.79760
[180]	train-auc:0.80908	eval-auc:0.79760
[181]	train-auc:0.80920	eval-auc:0.79761
[182]	train-auc:0.80933	eval-auc:0.79759
[183]	train-auc:0.80944	eval-auc:0.79758
[184]	train-auc:

[360]	train-auc:0.83714	eval-auc:0.79772
[361]	train-auc:0.83734	eval-auc:0.79771
[362]	train-auc:0.83756	eval-auc:0.79770
[363]	train-auc:0.83771	eval-auc:0.79770
[364]	train-auc:0.83790	eval-auc:0.79770
[365]	train-auc:0.83809	eval-auc:0.79769
[366]	train-auc:0.83820	eval-auc:0.79768
[367]	train-auc:0.83838	eval-auc:0.79770
[368]	train-auc:0.83860	eval-auc:0.79769
[369]	train-auc:0.83880	eval-auc:0.79768
[370]	train-auc:0.83898	eval-auc:0.79767
[371]	train-auc:0.83916	eval-auc:0.79769
[372]	train-auc:0.83933	eval-auc:0.79768
[373]	train-auc:0.83953	eval-auc:0.79768
[374]	train-auc:0.83969	eval-auc:0.79769
[375]	train-auc:0.83985	eval-auc:0.79768
[376]	train-auc:0.84001	eval-auc:0.79769
[377]	train-auc:0.84032	eval-auc:0.79768
[378]	train-auc:0.84050	eval-auc:0.79768
[379]	train-auc:0.84071	eval-auc:0.79770
[380]	train-auc:0.84095	eval-auc:0.79769
[381]	train-auc:0.84110	eval-auc:0.79769
[382]	train-auc:0.84128	eval-auc:0.79768
[383]	train-auc:0.84147	eval-auc:0.79768
[384]	train-auc:

[560]	train-auc:0.87167	eval-auc:0.79768
[561]	train-auc:0.87180	eval-auc:0.79767
[562]	train-auc:0.87199	eval-auc:0.79769
[563]	train-auc:0.87210	eval-auc:0.79769
[564]	train-auc:0.87226	eval-auc:0.79770
[565]	train-auc:0.87238	eval-auc:0.79769
[566]	train-auc:0.87249	eval-auc:0.79768
[567]	train-auc:0.87260	eval-auc:0.79767
[568]	train-auc:0.87281	eval-auc:0.79767
[569]	train-auc:0.87296	eval-auc:0.79767
[570]	train-auc:0.87313	eval-auc:0.79766
[571]	train-auc:0.87331	eval-auc:0.79766
[572]	train-auc:0.87343	eval-auc:0.79766
[573]	train-auc:0.87361	eval-auc:0.79765
[574]	train-auc:0.87376	eval-auc:0.79765
[575]	train-auc:0.87393	eval-auc:0.79763
[576]	train-auc:0.87402	eval-auc:0.79764
[577]	train-auc:0.87410	eval-auc:0.79764
[578]	train-auc:0.87427	eval-auc:0.79765
[579]	train-auc:0.87442	eval-auc:0.79766
[580]	train-auc:0.87458	eval-auc:0.79766
[581]	train-auc:0.87472	eval-auc:0.79766
[582]	train-auc:0.87484	eval-auc:0.79765
[583]	train-auc:0.87493	eval-auc:0.79763
[584]	train-auc:

[760]	train-auc:0.89606	eval-auc:0.79757
[761]	train-auc:0.89623	eval-auc:0.79756
[762]	train-auc:0.89635	eval-auc:0.79756
[763]	train-auc:0.89649	eval-auc:0.79756
[764]	train-auc:0.89659	eval-auc:0.79756
[765]	train-auc:0.89664	eval-auc:0.79756
[766]	train-auc:0.89681	eval-auc:0.79756
[767]	train-auc:0.89700	eval-auc:0.79755
[768]	train-auc:0.89713	eval-auc:0.79755
[769]	train-auc:0.89732	eval-auc:0.79754
[770]	train-auc:0.89754	eval-auc:0.79755
[771]	train-auc:0.89766	eval-auc:0.79756
[772]	train-auc:0.89775	eval-auc:0.79754
[773]	train-auc:0.89786	eval-auc:0.79754
[774]	train-auc:0.89802	eval-auc:0.79754
[775]	train-auc:0.89808	eval-auc:0.79755
[776]	train-auc:0.89815	eval-auc:0.79755
[777]	train-auc:0.89824	eval-auc:0.79754
[778]	train-auc:0.89838	eval-auc:0.79755
[779]	train-auc:0.89851	eval-auc:0.79756
[780]	train-auc:0.89862	eval-auc:0.79755
[781]	train-auc:0.89870	eval-auc:0.79755
[782]	train-auc:0.89879	eval-auc:0.79755
[783]	train-auc:0.89889	eval-auc:0.79754
[784]	train-auc:

[960]	train-auc:0.91957	eval-auc:0.79731
[961]	train-auc:0.91973	eval-auc:0.79732
[962]	train-auc:0.91982	eval-auc:0.79732
[963]	train-auc:0.91992	eval-auc:0.79732
[964]	train-auc:0.92005	eval-auc:0.79731
[965]	train-auc:0.92012	eval-auc:0.79731
[966]	train-auc:0.92029	eval-auc:0.79729
[967]	train-auc:0.92034	eval-auc:0.79730
[968]	train-auc:0.92041	eval-auc:0.79730
[969]	train-auc:0.92052	eval-auc:0.79730
[970]	train-auc:0.92062	eval-auc:0.79731
[971]	train-auc:0.92076	eval-auc:0.79730
[972]	train-auc:0.92089	eval-auc:0.79729
[973]	train-auc:0.92096	eval-auc:0.79729
[974]	train-auc:0.92106	eval-auc:0.79728
[975]	train-auc:0.92114	eval-auc:0.79726
[976]	train-auc:0.92120	eval-auc:0.79726
[977]	train-auc:0.92134	eval-auc:0.79726
[978]	train-auc:0.92139	eval-auc:0.79726
[979]	train-auc:0.92147	eval-auc:0.79726
[980]	train-auc:0.92164	eval-auc:0.79725
[981]	train-auc:0.92179	eval-auc:0.79725
[982]	train-auc:0.92198	eval-auc:0.79724
[983]	train-auc:0.92209	eval-auc:0.79724
[984]	train-auc:

In [51]:
pred_1a = []
for a in np.array(pred_train_1a):
    pred_1a.append(a)

len(pred_1a)

307507

In [52]:
### 予測値を特徴量としてデータフレームを作成
sk_id_curr = app_train['SK_ID_CURR'].copy().to_list()
train_x_2 = pd.DataFrame({'SK_ID_CURR': sk_id_curr, 'set4x': pred_1a})
print(train_x_2.shape)
train_x_2.head()

(307507, 2)


,SK_ID_CURR,set4x
0,100002,0.207998
1,100003,0.013902
2,100004,0.034973
3,100006,0.040586
4,100007,0.036123


In [53]:
train_x_2.to_csv(
    path_or_buf="./home-credit-default-risk/exports/train_set4xgb.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

In [54]:
pred_1a = []
for a in np.array(pred_test_1a):
    pred_1a.append(a)

#pred_1b = np.array(pred_1b)
len(pred_1a)

48744

In [55]:
### 予測値を特徴量としてデータフレームを作成
sk_id_curr = app_test['SK_ID_CURR'].copy().to_list()
test_x_2 = pd.DataFrame({'SK_ID_CURR': sk_id_curr, 'set4x': pred_1a})
print(test_x_2.shape)
test_x_2.head()

(48744, 2)


,SK_ID_CURR,set4x
0,100001,0.034978
1,100005,0.118268
2,100013,0.039774
3,100028,0.037283
4,100038,0.164890


In [56]:
test_x_2.to_csv(
    path_or_buf="./home-credit-default-risk/exports/test_set4xgb.csv" # 出力先
    ,index=None                                        # indexの出力有無
)